In [1]:
from dotenv import load_dotenv
import sys

import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.vectorstores import FAISS

sys.path.append('..')
from backend.DB.db import Database

c:\Users\karen\Anaconda3\envs\hotel-reviews\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.9) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
load_dotenv()
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)


In [3]:
db = Database()
reviews = db.query_sql('SELECT title, review_text FROM raw_reviews')
texts = [f'{title}. {text}' for title, text in list(reviews)]
docs = text_splitter.create_documents(texts)
db = FAISS.from_documents(documents=docs, embedding=embeddings)


In [4]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k":2})
)

tools = [
    Tool(name='Retrival QA System',
         func=retrieval_qa.run,
         description="Answering questions")
]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [5]:
response = agent.run("Tell me about reviews that talk about bugs?")
print(response)



> Entering new  chain...
 I need to find reviews that mention bugs.
Action: Retrival QA System
Action Input: Reviews that mention bugs
Observation:  Reviews that mention bugs may be referring to the unpleasant stay at Gonzo Inn, Moab, Utah, where the room had bugs and the front desk was not accommodating.
Thought: I now know the final answer.
Final Answer: Reviews that mention bugs may be referring to the unpleasant stay at Gonzo Inn, Moab, Utah, where the room had bugs and the front desk was not accommodating.

> Finished chain.
Reviews that mention bugs may be referring to the unpleasant stay at Gonzo Inn, Moab, Utah, where the room had bugs and the front desk was not accommodating.


In [6]:
response = agent.run("What do reviews say about breakfast?")
print(response)



> Entering new  chain...
 I should look for reviews about breakfast.
Action: Retrival QA System
Action Input: Reviews about breakfast
Observation:  The reviews about breakfast were generally positive. The breakfast was limited but the food was good. The $14 voucher was nice but it didn't cover a meal and a coffee. It was still only a few bucks more which people couldn't complain about.
Thought: I now know the final answer.
Final Answer: Reviews about breakfast were generally positive. The breakfast was limited but the food was good. The $14 voucher was nice but it didn't cover a meal and a coffee. It was still only a few bucks more which people couldn't complain about.

> Finished chain.
Reviews about breakfast were generally positive. The breakfast was limited but the food was good. The $14 voucher was nice but it didn't cover a meal and a coffee. It was still only a few bucks more which people couldn't complain about.


In [7]:
response = agent.run("Can you summarize bad reviews?")
print(response)



> Entering new  chain...
 I need to find a way to summarize the reviews.
Action: Retrival QA System
Action Input: Summarize bad reviews
Observation:  The reviews for this room were not good. People mentioned that the janitorial services were not up to par and that the window coverings were not adequate for blocking out the residents in the back of the room.
Thought: I need to find a way to condense this information.
Action: Retrival QA System
Action Input: Summarize bad reviews
Observation:  The reviews for this room were not good. People mentioned that the janitorial services were not up to par and that the window coverings were not adequate for blocking out the residents in the back of the room.
Thought: I now know the final answer.
Final Answer: The reviews for this room were not good, with people citing issues with janitorial services and window coverings not blocking out the residents in the back of the room.

> Finished chain.
The reviews for this room were not good, with peopl

In [8]:
response = agent.run("What is the average sentiment about the cleanliness of rooms?")
print(response)



> Entering new  chain...
 I need to find out what people are saying about the cleanliness of rooms.
Action: Retrival QA System
Action Input: Reviews about the cleanliness of rooms
Observation:  Reviews about the cleanliness of rooms have been positive, with people noting that the rooms were clean and comfortable.
Thought: I need to calculate the average sentiment of the reviews.
Action: Retrival QA System
Action Input: Average sentiment of reviews about the cleanliness of rooms
Observation:  Positive
Thought: I now know the final answer.
Final Answer: The average sentiment about the cleanliness of rooms is positive.

> Finished chain.
The average sentiment about the cleanliness of rooms is positive.


In [9]:
response = agent.run("Were there any complaints were there about the cleanliness of rooms?")
print(response)



> Entering new  chain...
 I should look for reviews or feedback from customers.
Action: Retrival QA System
Action Input: Reviews of the cleanliness of rooms
Observation:  Reviews of the cleanliness of rooms have been positive, with guests noting that the rooms were clean and comfortable.
Thought: I now know the final answer
Final Answer: Reviews of the cleanliness of rooms have been positive, with guests noting that the rooms were clean and comfortable.

> Finished chain.
Reviews of the cleanliness of rooms have been positive, with guests noting that the rooms were clean and comfortable.


In [10]:
response = agent.run("What aspects of stays did receive complaints?")
print(response)



> Entering new  chain...
 I need to find out what people have complained about in regards to stays.
Action: Retrival QA System
Action Input: What complaints have been made about stays?
Observation:  Complaints have been made about the janitorial rating, the window coverings, the patio not being swept, and being placed next to an adjoining room with a pet and louder guests.
Thought: I now know the final answer
Final Answer: People have complained about the janitorial rating, the window coverings, the patio not being swept, and being placed next to an adjoining room with a pet and louder guests.

> Finished chain.
People have complained about the janitorial rating, the window coverings, the patio not being swept, and being placed next to an adjoining room with a pet and louder guests.


In [11]:
response = agent.run("Can you provide an overall summary of the reviews?")
print(response)



> Entering new  chain...
 I need to find a way to summarize the reviews.
Action: Retrival QA System
Action Input: Summarize reviews
Observation:  Reviews of the experience were positive, and the reviewer would stay there again.
Thought: I now know the final answer
Final Answer: The reviews of the experience were generally positive, and the reviewer would stay there again.

> Finished chain.
The reviews of the experience were generally positive, and the reviewer would stay there again.
